<a href="https://colab.research.google.com/github/yootaka/AiQuest/blob/main/efficientnetv2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考

https://github.com/dajiro-repo/blog/blob/master/notebook/blog_timm.ipynb

https://dajiro.com/entry/2020/07/24/161040

In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils as utils
import torchvision.utils as vutils
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torch.optim as optim
import os

ライブラリの読み込み

In [4]:
!pip install timm
import timm

     |████████████████████████████████| 376 kB 5.3 MB/s 


In [5]:
# Google ドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
X = []
Y = []

folder_path = "/content/drive/My Drive/Colab Notebooks/AiQuest/ihara/A/"
file_list = os.listdir(folder_path)

In [7]:
print(file_list) #ファイル確認

['No.1-A1.bmp', 'No.1-A2.bmp', 'No.10-A2.bmp', 'No.10-A1.bmp', 'No.11-A2.bmp', 'No.12-A1.bmp', 'No.11-A1.bmp', 'No.13-A1.bmp', 'No.12-A2.bmp', 'No.13-A2.bmp', 'No.14-A1.bmp', 'No.15-A1.bmp', 'No.14-A2.bmp', 'No.16-A1.bmp', 'No.15-A2.bmp', 'No.17-A1.bmp', 'No.16-A2.bmp', 'No.17-A2.bmp', 'No.18-A2.bmp', 'No.18-A1.bmp', 'No.19-A1.bmp', 'No.19-A2.bmp', 'No.2-A1.bmp', 'No.2-A2.bmp', 'No.20-A1.bmp', 'No.21-A2.bmp', 'No.21-A1.bmp', 'No.20-A2.bmp', 'No.23-A1.bmp', 'No.22-A1.bmp', 'No.22-A2.bmp', 'No.24-A1.bmp', 'No.24-A2.bmp', 'No.23-A2.bmp', 'No.25-A2.bmp', 'No.25-A1.bmp', 'No.26-A1.bmp', 'No.26-A2.bmp', 'No.27-A1.bmp', 'No.27-A2.bmp', 'No.28-A1.bmp', 'No.28-A2.bmp', 'No.29-A1.bmp', 'No.3-A1.bmp', 'No.29-A2.bmp', 'No.3-A2.bmp', 'No.30-A1.bmp', 'No.30-A2.bmp', 'No.31-A1.bmp', 'No.31-A2.bmp', 'No.32-A2.bmp', 'No.32-A1.bmp', 'No.33-A1.bmp', 'No.34-A1.bmp', 'No.33-A2.bmp', 'No.34-A2.bmp', 'No.35-A1.bmp', 'No.35-A2.bmp', 'No.36-A2.bmp', 'No.36-A1.bmp', 'No.37-A2.bmp', 'No.37-A1.bmp', 'No.38-A1.bmp

In [8]:
###############
# 入力パラメータ
###############
#モデル名
model_name = 'tf_efficientnetv2_s_in21ft1k'
#画像のパス
image_path = folder_path
#バッチサイズ
batch_size = 32
#エポック数
epochs = 5
#GPUID
ngpu = 1
#ラベル数
n_class = 1000
#画像サイズ
image_size = 384
#ワーカー数
workers = 1
#訓練用データ比率
train_ratio = 0.7
#検証用データ比率
val_ratio = 0.2
#学習率
lr = 0.001
#Adamのbeta1
beta1 = 0.9
#チャネル数
nc = 3
#中間層チャネル数のユニット
ndf = 16
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
#if not os.path.exists('./imagenetv2-matched-frequency-format-val/'):
#    !curl -s https://s3-us-west-2.amazonaws.com/imagenetv2public/imagenetv2-matched-frequency.tar.gz | tar x
# dataset = Dataset('./imagenetv2-matched-frequency-format-val/')
# for i in range(len(dataset)): # warmup
#     dummy = dataset[i]

In [9]:
def get_data_loaders(batch_size, conf, train_ratio, val_ratio):
    dataset = dset.ImageFolder(root=image_path,
                               transform=transforms.Compose([
                                   transforms.Resize(image_size),
                                   transforms.CenterCrop(image_size),
                                   transforms.ToTensor(),
                                   transforms.Normalize(conf["mean"], conf["std"]),
                               ]))

    num_data = len(dataset)
    num_list = [int(train_ratio * num_data),
                int(val_ratio * num_data),
                num_data -
                int(train_ratio * num_data) -
                int(val_ratio * num_data)]
    collate_fn = None
    train_data, val_data, test_data = utils.data.random_split(dataset,
                                                              [num_list[0],
                                                               num_list[1],
                                                               num_list[2]])
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                             shuffle=True, num_workers=workers,
                                             collate_fn = collate_fn)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size,
                                             shuffle=True, num_workers=workers,
                                             collate_fn = collate_fn)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
                                             shuffle=True, num_workers=workers,
                                             collate_fn = collate_fn)
    
    return train_loader, val_loader, test_loader

In [10]:

data_config = timm.data.resolve_data_config({}, model=model_name, verbose=True)

class EfficientNet_b0(nn.Module):
    def __init__(self, n_out):
        super(EfficientNet_b0, self).__init__()
        #モデルの定義
        self.effnet = timm.create_model(model_name, pretrained=True)
        #最終層の再定義
        self.effnet.classifier = nn.Linear(1280, n_out)

    def forward(self, x):
        return self.effnet(x)

In [11]:
#モデル
model = EfficientNet_b0(n_class)
params_to_update = []
update_param_names = ["effnet.classifier.weight", "effnet.classifier.bias"]
for name, param in model.named_parameters():
    if name in update_param_names:
      param.requires_grad = True
      params_to_update.append(param)
    else:
      param.requires_grad = False

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-effv2-weights/tf_efficientnetv2_s_21ft1k-d7dafa41.pth" to /root/.cache/torch/hub/checkpoints/tf_efficientnetv2_s_21ft1k-d7dafa41.pth


In [12]:
# 損失関数（closs entropy）
criterion = nn.CrossEntropyLoss()
#Optimizer(Adamを使用)
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(beta1, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [13]:
train_loader, val_loader, test_loader = get_data_loaders(batch_size,
                                                         data_config,
                                                         train_ratio,
                                                         val_ratio)
dataloaders_dict = {"train":train_loader, "val":val_loader}

real_batch = next(iter(train_loader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0][:4], padding=2, normalize=True).cpu(),(1,2,0)))

FileNotFoundError: ignored

In [ ]:
accuracy, accuracy_val = [], []
model.to(device)
for epoch in range(epochs):

    for phase in ["train", "val"]:
        if phase == "train":
          model.train()
        else:
          model.eval()

        loss_epoch = 0.0
        acc_epoch = 0.0

        if (epoch == 0) and (phase == "train"):
            continue

        for inputs, labels in dataloaders_dict[phase]:
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase == "train"):
                labels = labels.to(device)
                inputs = inputs.to(device)
                outputs = model(inputs)
                
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                
                if phase == "train":
                    loss.backward()
                    optimizer.step()
                
                loss_epoch += loss.item() * inputs.size(0)
                acc_epoch += torch.sum(preds == labels.data)
        
        loss_epoch = loss_epoch / len(dataloaders_dict[phase].dataset)
        acc_epoch = acc_epoch.double() / len(dataloaders_dict[phase].dataset)
        print(f"phase: {phase}",
              f"LR: {scheduler.get_last_lr()[0]} "
              f"epoch: {epoch}",
              f"loss: {loss_epoch:.4f}",
              f"accuracy: {acc_epoch:.4f}")
             # f"accuracy: {acc_epoch}:.4f}")
        if phase == "train":
            scheduler.step()

print('Finished Training')

phase: val LR: 0.001 epoch: 0 loss: 7.0726 accuracy: 0.0005
phase: train LR: 0.001 epoch: 1 loss: 4.6999 accuracy: 0.2970
phase: val LR: 0.001 epoch: 1 loss: 2.7684 accuracy: 0.5455
phase: train LR: 0.001 epoch: 2 loss: 0.4368 accuracy: 0.9346
